# An example of finite volume method for heat equation in polar coordinates

A Jupyter Notebook with source code is located at [this](https://github.com/krashkov/Blog) GitHub repository.

**Contents**

* The basics of FVM
* Numerical example: heat transfer problem in polar coordinates
    * Analytical solution
    * Numerical solution with FVM

In [11]:
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.colors as colors

from scipy.integrate import odeint
from scipy.sparse    import diags

---

## 1 The basics of FVM

Finite volume method is a universal tool for constructing *conservative* schemes which are based even on unstructured meshes. Consider the following equation in divergence form:

$$
\frac{\partial u}{\partial t} + \frac{\partial f}{\partial x_1} + \frac{\partial f}{\partial x_2} = g(u)
$$

Integrating over arbitrary volume $V$ and applying the Divergence theorem one may obtain integral form of conservation law:

$$
\int_V \frac{\partial u}{\partial t} dV + \int_{\partial V} (\mathbf{f}, \mathbf{n}) dS = \int_V g(u) dV
$$

The main idea of FVM is to approximate this conservation law. Suppose that the computational domain $\Omega$ is divided into elements $V_i$ so that $\Omega = \cup_i V_i$. For each cell let us introduce the average value $u_i = \frac{1}{V_i} \int_{V_i} udV$. Note that $u_i$ is associated with center of $i$-th cell. Conservation law for each cell can be written as: 

$$
\frac{\partial u_i}{\partial t} + \frac{1}{V_i} \int_{\partial V_i} (\mathbf{f}, \mathbf{n}) dS = \frac{1}{V_i} \int_{V_i} g(u) dV
$$

Surface integral is equal to sum of integrals over all $N_i$ faces (edges) of $i$-th cell:

$$
\int_{\partial V_i} (\mathbf{f}, \mathbf{n}) dS = \sum_{i = 1}^{N_i} \int_{S_{\alpha}} (\mathbf{f}, \mathbf{n}) dS
$$

In most cases one may suppose that:

$$
\begin{aligned}
\frac{1}{V_i}\int_{V_i} g(u) dV &\approx g(u_i)\\
\int_{S_{\alpha}} (\mathbf{f}, \mathbf{n}) dS &\approx (\mathbf{f}_{\alpha}, \mathbf{n}_{\alpha}) S_{\alpha}
\end{aligned}
$$

where $S_{\alpha}$ is a surface area of $\alpha$-th face (length of $\alpha$-th edge), $\mathbf{n}_{\alpha}$ is an outer-pointing normal vector to $\alpha$-th face (edge) and $\mathbf{f}_{\alpha}$ is associated with center of $\alpha$-th face (edge). As a result, finite volume scheme takes the following form:

$$
\frac{\partial u_i}{\partial t} + \frac{1}{V_i} \sum_{\alpha = 1}^{N_i} (\mathbf{f}_{\alpha}, \mathbf{n}_{\alpha}) S_{\alpha} = g(u_i)
$$

It remains to specify the way of discretization of $\frac{\partial u_i}{\partial t}$ and calculation of $\mathbf{f}_{\alpha}$ in terms of $u_i$.

---

## 2 Numerical example: heat transfer problem in polar coordinates

Consider the following heat equation:

$$
\begin{cases}
c\rho\cfrac{\partial T}{\partial t} - \cfrac{1}{r}\cfrac{\partial}{\partial r}\left(kr\cfrac{\partial T}{\partial r}\right)=q,\ 0<r<a\\
T(0, r) = T_0(r)\\
\cfrac{\partial T}{\partial r}(t, 0) = 0\\
\left(k\cfrac{\partial T}{\partial r} + \alpha (T - T_{\infty}) + \sigma T^4\right)(t,a) = 0
\end{cases}
\tag{1}
$$

### 2.2 Numerical solution with FVM



---

*References*

1. K. Gustavsson. Lecture Notes: Numerical Treatment of Differential Equations, KTH Institute of Technology [[link](http://www.csc.kth.se/utbildning/kth/kurser/DN2255/ndiff13/)]